## Load data

In [1]:
## Load in dependencies
import pandas as pd,\
  seaborn as sns,\
  matplotlib.pyplot as pypl,\
  datetime as dt


In [2]:
## Create functions for assigning periods, assigning control/treatment
def class_by_trading_ratios(mean_ratio,rank_ratio):
  if rank_ratio <= 10:
    return 'top_10'
  elif mean_ratio >= 0.03:
    return 'treatment'
  else:
    return 'control'

def class_by_periods(string_date, date_0, date_1 ):
  if string_date <=  date_0  :
    return 'before_rule'
  elif string_date >=  date_1 :
    return 'post_rule'
  else:
    return 'other'

In [3]:
## Set variables for discussion periods
discussion_start = dt.datetime(2009, 7, 1)
discussion_end = dt.datetime(2010, 7, 1)

In [5]:
## Load in data, convert to proper date type
df_input = pd.read_csv(
  './DiD_data.csv',
  header='infer',
  index_col=False
)

df_input.dropna(inplace=True)

df_input['date_parsed'] =  pd.to_datetime(df_input['rssd9999'], format='%Y%m%d')

In [6]:
## Set color palette, includes gridlines
pypl.style.use('bmh')

-----

## Get groups

In [ ]:
## Get average of mean trading ratios per bank, in pre discussion period, broadcast to original dataset
df_top_10_ratio = df_input[
  df_input['date_parsed'] < '2007-01-01'  
].groupby(
  ['rssd9001']
)['bhc_avgtradingratio'].mean().reset_index()

df_top_10_ratio.rename(columns={"bhc_avgtradingratio": "pre_2007_mean_ratio"}, inplace=True)

df_top_10_ratio['mean_ratio_rank'] = df_top_10_ratio['pre_2007_mean_ratio'].rank(ascending=False,method='min')

df_merged = df_input.merge(right=df_top_10_ratio,on='rssd9001' ,how='left' )

In [ ]:
##

df_merged['bank_group'] = df_merged.apply(lambda x: class_by_trading_ratios( x['pre_2007_mean_ratio'], x['mean_ratio_rank'])  , axis=1)

-----

## Visualise

In [ ]:
sns.lineplot(
  data= df_merged,
  x='date_parsed',
  y='bhc_avgtradingratio',
  hue='bank_group'
)

pypl.title('Per-Period Mean Trading Ratio, with Volcker Rule period - Treatment, Control, Top 10')

pypl.xlabel('Period')

pypl.ylabel('Mean Trading Ratio')

pypl.gcf().set_size_inches(15, 9)

pypl.vlines(x=discussion_start  ,ymin=0,ymax=0.33)
pypl.vlines(x= discussion_end ,ymin=0,ymax=0.33)

pypl.savefig('./line_quarterly_trading.png')

#pypl.show()

pypl.close('all')

-----

## Descriptive Statistics

In [ ]:
df_merged['date_group'] = df_merged.apply(
  lambda x: class_by_periods( x['date_parsed'],discussion_start,discussion_end), 
  axis=1
)

In [ ]:
df_period_ratio = df_merged.groupby(['date_group','rssd9001','bank_group'])['bhc_avgtradingratio'].mean().reset_index()

In [ ]:
sns.kdeplot(
  data=df_period_ratio[ df_period_ratio['bank_group']== 'treatment' ],
  x='bhc_avgtradingratio',
  hue='date_group',
#  element='step',
#  stat='count'
)

pypl.title('Density Plot - Average of Per-Bank Mean Trading Ratio, Before/After Rule; Treatment')

pypl.xlabel('Average of Mean Trading Ratio')

pypl.ylabel('Density')

pypl.gcf().set_size_inches(15, 9)

pypl.savefig('./density_trading_treatment.png')

#pypl.show()

pypl.close('all')

In [ ]:
sns.kdeplot(
  data=df_period_ratio[ df_period_ratio['bank_group']== 'top_10' ],
  x='bhc_avgtradingratio',
  hue='date_group',
#  element='step',
#  stat='count'
)

pypl.title('Density Plot - Average of Per-Bank Mean Trading Ratio, Before/After Rule; Top 10')

pypl.xlabel('Average of Mean Trading Ratio')

pypl.ylabel('Density')

pypl.gcf().set_size_inches(15, 9)

pypl.savefig('./density_trading_top10.png')

#pypl.show()

pypl.close('all')

In [ ]:
describe_columns= df_input.columns.tolist()
describe_columns.remove('rssd9001')
describe_columns.remove('rssd9999')
describe_columns.remove('after_DFA_1')
describe_columns.remove('treat_3_b_avg')
describe_columns.remove('dep_cpp_bankquarter')

In [ ]:
df_input[describe_columns].describe().transpose().to_html('./full_stats.html')

## Generate Pairplot

In [9]:
describe_columns= df_input.columns.tolist()
describe_columns.remove('rssd9001')
describe_columns.remove('rssd9999')
describe_columns.remove('after_DFA_1')
#describe_columns.remove('treat_3_b_avg')
describe_columns.remove('bhc_avgtradingratio')
describe_columns.remove('dep_cpp_bankquarter')

In [11]:
sns.pairplot(
  data = df_input[describe_columns],
  hue='treat_3_b_avg'
)

#pypl.show()
pypl.savefig('./pairplot_covariate_treatment.png')

pypl.close('all')
